<a href="https://colab.research.google.com/github/jojohnathon/job-market-analysis/blob/main/ai_investment_impact_on_swe_market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
gvyshnya_forbes_ai50_2023_path = kagglehub.dataset_download('gvyshnya/forbes-ai50-2023')
saeedsarrafzadeh_forbes_ai50_2024_path = kagglehub.dataset_download('saeedsarrafzadeh/forbes-ai50-2024')
arshkon_linkedin_job_postings_path = kagglehub.dataset_download('arshkon/linkedin-job-postings')

print('Data source import complete.')


100%|██████████| 4.36k/4.36k [00:00<00:00, 5.34MB/s]

Extracting files...


100%|██████████| 1.55k/1.55k [00:00<00:00, 2.36MB/s]

Extracting files...


100%|██████████| 159M/159M [00:01<00:00, 113MB/s]

Extracting files...


Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#load data into pandas
job_postings = pd.read_csv(arshkon_linkedin_job_postings_path+"/postings.csv")
ai_2024 = pd.read_csv(saeedsarrafzadeh_forbes_ai50_2024_path+'/AI50 2024.csv')
ai_2023 = pd.read_csv(gvyshnya_forbes_ai50_2023_path+"/AI50-2023.csv")

In [10]:
ai_2024.head()
ai_2023.head()
job_postings.head()


,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0


Normalize ai keywords in job postings by word count

In [13]:
ai_keywords = [
    "artificial intelligence", "machine learning", "deep learning", "neural networks",
    "GPT", "ChatGPT", "LLM", "computer vision", "NLP", "reinforcement learning",
    "AutoML", "generative AI", "transformers", "BERT", "Copilot", "chatbot", "speech recognition"
]

def count_ai_keywords(text):
    count = 0
    for keyword in ai_keywords:
        if keyword in text.lower():
            count += 1
    return count

job_postings["wordcount"] = job_postings["description"].apply(lambda x: len(str(x).split()))
job_postings['wordcount']

,wordcount
0,358
1,492
2,66
3,209
4,36
...,...
123844,297
123845,776
123846,924
123847,484
